In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
import random

In [2]:
# Load Dataset
def load_data():
    # Replace with actual dataset loading logic
    input_texts = ["Hi", "How are you?", "What is your name?", "Goodbye"]
    target_texts = ["Hello", "I'm fine, thank you!", "I am a chatbot", "See you later"]
    return input_texts, target_texts

In [3]:
# Preprocessing
def preprocess_data(input_texts, target_texts, vocab_size, max_sequence_length):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
    tokenizer.fit_on_texts(input_texts + target_texts)
    
    input_sequences = tokenizer.texts_to_sequences(input_texts)
    target_sequences = tokenizer.texts_to_sequences(target_texts)
    
    input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
    target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')

    return input_sequences, target_sequences, tokenizer

In [4]:
# Define Basic LSTM Model
def build_basic_seq2seq_model(vocab_size, embedding_dim, max_sequence_length):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
        LSTM(128, return_sequences=True),
        Dropout(0.5),
        LSTM(128),
        Dropout(0.5),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# Define Attention-based Seq2Seq Model
def build_attention_seq2seq_model(vocab_size, embedding_dim, max_sequence_length):
    # Encoder
    encoder_inputs = Input(shape=(max_sequence_length,))
    encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm, state_h, state_c = LSTM(128, return_state=True)(encoder_embedding)

    # Decoder
    decoder_inputs = Input(shape=(max_sequence_length,))
    decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(128, return_sequences=True, return_state=False)(decoder_embedding, initial_state=[state_h, state_c])
    attention = Dense(1, activation='tanh')(decoder_lstm)
    decoder_outputs = Dense(vocab_size, activation='softmax')(attention)

    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
# Train Model
def train_model(model, X_train, Y_train, epochs, batch_size):
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return history

In [7]:
# Evaluate Model
def evaluate_model(model, tokenizer, test_input, test_target, max_sequence_length):
    bleu_scores = []
    for input_text, target_text in zip(test_input, test_target):
        input_seq = tokenizer.texts_to_sequences([input_text])
        input_seq = pad_sequences(input_seq, maxlen=max_sequence_length)
        
        prediction = model.predict(input_seq)
        predicted_text = tokenizer.sequences_to_texts(np.argmax(prediction, axis=-1))

        bleu_scores.append(sentence_bleu([target_text.split()], predicted_text.split()))

    return np.mean(bleu_scores)

In [8]:
# Visualization
def plot_training(history1, history2):
    plt.figure(figsize=(12, 6))

    # Basic Model
    plt.plot(history1.history['loss'], label='Basic Model - Loss')
    plt.plot(history1.history['val_loss'], label='Basic Model - Val Loss')

    # Attention Model
    plt.plot(history2.history['loss'], label='Attention Model - Loss')
    plt.plot(history2.history['val_loss'], label='Attention Model - Val Loss')

    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [9]:
# Main Workflow
input_texts, target_texts = load_data()
vocab_size = 5000
embedding_dim = 128
max_sequence_length = 10

X_data, Y_data, tokenizer = preprocess_data(input_texts, target_texts, vocab_size, max_sequence_length)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

In [11]:
# Basic Model
basic_model = build_basic_seq2seq_model(vocab_size, embedding_dim, max_sequence_length)
print("Training Basic Model...")
history_basic = train_model(basic_model, X_train, Y_train, epochs=10, batch_size=32)

Training Basic Model...
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\Metaverse_KPMG\anaconda3\envs\chatbot-env\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Metaverse_KPMG\anaconda3\envs\chatbot-env\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Metaverse_KPMG\anaconda3\envs\chatbot-env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\Metaverse_KPMG\anaconda3\envs\chatbot-env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\Metaverse_KPMG\anaconda3\envs\chatbot-env\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\Metaverse_KPMG\anaconda

In [12]:
# Attention Model
attention_model = build_attention_seq2seq_model(vocab_size, embedding_dim, max_sequence_length)
print("Training Attention Model...")
history_attention = train_model(attention_model, [X_train, X_train], Y_train, epochs=10, batch_size=32)

Training Attention Model...
Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 8.5172 - accuracy: 0.0000e+00 - val_loss: 8.5153 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 43ms/step - loss: 8.5141 - accuracy: 0.0500 - val_loss: 8.5133 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 51ms/step - loss: 8.5108 - accuracy: 0.0500 - val_loss: 8.5111 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 51ms/step - loss: 8.5072 - accuracy: 0.0500 - val_loss: 8.5086 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 50ms/step - loss: 8.5031 - accuracy: 0.0500 - val_loss: 8.5059 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 52ms/step - loss: 8.4984 - accuracy: 0.0500 - val_loss: 8.5027 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 52ms/step - loss: 8.4931 - accuracy: 0.0500 - val_loss: 8.4

In [ ]:
# Plot Comparison
plot_training(history_basic, history_attention)

# Evaluate
print("Evaluating Basic Model...")
basic_bleu = evaluate_model(basic_model, tokenizer, X_test, Y_test, max_sequence_length)

print("Evaluating Attention Model...")
attention_bleu = evaluate_model(attention_model, tokenizer, X_test, Y_test, max_sequence_length)

print(f"Basic Model BLEU Score: {basic_bleu}")
print(f"Attention Model BLEU Score: {attention_bleu}")